### Conexión a Confluent y creación de topic

In [0]:
# dbutils.fs.put("dbfs:/FileStore/confluent_client_properties", """
#bootstrap.servers=pkc-03vj5.europe-west8.gcp.confluent.cloud:9092
#security.protocol=SASL_SSL
#sasl.mechanism=PLAIN
#sasl.username= eliminado por seguridad
#sasl.password= eliminado por seguridad
#""", overwrite=True)

Wrote 225 bytes.


True

In [0]:
def read_config():
  config = {}
  with open("/dbfs/FileStore/confluent_client_properties") as fh:
    for line in fh:
      line = line.strip()
      if len(line) != 0 and line[0] != "#":
        parameter, value = line.strip().split('=', 1)
        config[parameter] = value.strip()
  return config

  # mala práctica tener las claves en filestore, se puede acceder desde la url. Habría que haber usado databricks secrets

In [0]:
conf = read_config()

In [0]:
!pip install confluent_kafka

  Obtaining dependency information for confluent_kafka from https://files.pythonhosted.org/packages/40/aa/ebf3facd881cb0b4e79fae29f040079ff2cb1ae4aabee08456e3ab536828/confluent_kafka-2.11.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/3.9 MB ? eta -:--:--
   ━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/3.9 MB 10.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━ 3.8/3.9 MB 55.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 39.9 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
!pip install faker 

  Obtaining dependency information for faker from https://files.pythonhosted.org/packages/61/7d/8b50e4ac772719777be33661f4bde320793400a706f5eb214e4de46f093c/faker-37.6.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.9 MB ? eta -:--:--
   ━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2/1.9 MB 6.3 MB/s eta 0:00:01
   ━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.5/1.9 MB 7.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━ 0.9/1.9 MB 8.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━ 1.3/1.9 MB 9.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━ 1.7/1.9 MB 10.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.9/1.9 MB 10.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.3 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from confluent_kafka.admin import AdminClient, NewTopic
import random
import time
import json
from faker.providers import BaseProvider
from confluent_kafka import Producer 
from faker import Faker

admin_client = AdminClient(conf) 
topic_list = admin_client.list_topics().topics 
print("Topics en el cluster de Kafka:") 
topic_list 

Topics en el cluster de Kafka:


{'pedidos': TopicMetadata(pedidos, 3 partitions)}

In [0]:
topic = "pedidos"
#new_topic = NewTopic(topic, num_partitions=3, replication_factor=3)
#fs = admin_client.create_topics([new_topic])

In [0]:
topic_list = admin_client.list_topics().topics 
print(f"Topics en el cluster de Kafka:\n{topic_list}") 

Topics en el cluster de Kafka:
{'pedidos': TopicMetadata(pedidos, 3 partitions)}


### Simulación de stream

In [0]:
path_silver = 'abfss://datalake@dls0tfm.dfs.core.windows.net/silver/'

df_art = spark.read.format('delta').load(f'{path_silver}articulos')

In [0]:
df_art.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- preis1: double (nullable = true)
 |-- preis2: double (nullable = true)
 |-- preis3: double (nullable = true)
 |-- ean: integer (nullable = true)
 |-- wgr: integer (nullable = true)
 |-- processing_date: date (nullable = true)



In [0]:
bebidas = df_art.filter(df_art.wgr.isin(1, 2, 3, 4)).select('name')

In [0]:
comida = df_art.filter(df_art.wgr.isin(7, 8, 9)).select('name')

In [0]:
lista_bebidas = [bebida.name for bebida in bebidas.collect()][:-1]

In [0]:
lista_comidas = [plato.name for plato in comida.collect()]

In [0]:
class OrdenPuerta(BaseProvider):
    def comida(self,lista_comidas):
        return random.choice(lista_comidas) 
    def bebida(self, lista_bebidas):
        return random.choice(lista_bebidas)
    def mesa_cliente(self):
        mesas = [102, 103, 104, 105, 106, 107, 108, 201, 202, 203, 204, 300, 301, 302, 303, 301, 402, 403, 404, 405, 501, 502, 503, 504, 505, 3, 6]
        return random.choice(mesas)
    def produce_msg(self,FakerInstance, ordercount=1, max_com = 15, max_beb = 5):
        platos= []
        for plato in range(random.randint(1,max_com)):
            platos.append(FakerInstance.comida(lista_comidas))
        bebidas = []
        for drink in range(random.randint(1,max_beb)):
            bebidas.append(FakerInstance.bebida(lista_bebidas))
        mesa = FakerInstance.mesa_cliente()
        value = {
            "id": ordercount,
            "mesa": mesa,
            "comida": platos,
            "bebida": bebidas,
            "timestamp": int(time.time() * 1000)
        }
        key = str(mesa)
        return key, value

In [0]:
producer = Producer(conf) 
fake = Faker()
fake.add_provider(OrdenPuerta)
MAX_PLATOS_EN_COMANDA = 15
MAX_BEBIDAS_EN_COMANDA = 5
MAX_COMANDAS = 120

In [0]:
for i in range(MAX_COMANDAS):
    key,value = fake.produce_msg(
                    fake,
                    i+1,
                    MAX_PLATOS_EN_COMANDA,
                    MAX_BEBIDAS_EN_COMANDA,
                )
    producer.produce(topic=topic, key=key, value=json.dumps(value))
    producer.flush()
    print(f"comanda {i+1} pedida")
    # sleeping time
    sleep_time = (
        random.randint(0, int(1 * 10000)) / 10000
    )
    print("en pausa durante..." + str(sleep_time) + "s")
    time.sleep(sleep_time)

%6|1757589349.634|GETSUBSCRIPTIONS|rdkafka#producer-3| [thrd:main]: Telemetry client instance id changed from AAAAAAAAAAAAAAAAAAAAAA to CD7/GQpLQ5+ElABA/kjtJw


comanda 1 pedida
en pausa durante...0.8811s
comanda 2 pedida
en pausa durante...0.1031s
comanda 3 pedida
en pausa durante...0.316s
comanda 4 pedida
en pausa durante...0.212s
comanda 5 pedida
en pausa durante...0.2372s
comanda 6 pedida
en pausa durante...0.3814s
comanda 7 pedida
en pausa durante...0.5081s
comanda 8 pedida
en pausa durante...0.9449s
comanda 9 pedida
en pausa durante...0.4282s
comanda 10 pedida
en pausa durante...0.8898s
comanda 11 pedida
en pausa durante...0.32s
comanda 12 pedida
en pausa durante...0.1054s
comanda 13 pedida
en pausa durante...0.3302s
comanda 14 pedida
en pausa durante...0.1833s
comanda 15 pedida
en pausa durante...0.8076s
comanda 16 pedida
en pausa durante...0.9115s
comanda 17 pedida
en pausa durante...0.1752s
comanda 18 pedida
en pausa durante...0.6928s
comanda 19 pedida
en pausa durante...0.0768s
comanda 20 pedida
en pausa durante...0.0157s
comanda 21 pedida
en pausa durante...0.6646s
comanda 22 pedida
en pausa durante...0.7705s
comanda 23 pedida
en pa